In [80]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math as m
import mahotas as mt
import glob
import pandas as pd
cv2.__version__

'3.4.1'

In [2]:
path = glob.glob("C:/Users/suhas/Documents/Mini Project/Leaves/*.jpg")

## Create Columns and DataFrame

In [81]:
columns = ['common name','red_mean', 'red_var', 'blue_mean', 'blue_var', 'green_mean', 'green_var', 'area', 'perimeter', 'length', 'width', 'aspect ratio', 'form factor', 'rectangularity', 'narrow factor', 'perimeter ratio of diameter', 'perimeter ratio of length and width', 'contrast', 'correlation', 'entropy', 'inverse difference moments']

In [89]:
columns

['common name',
 'red_mean',
 'red_var',
 'blue_mean',
 'blue_var',
 'green_mean',
 'green_var',
 'area',
 'perimeter',
 'length',
 'width',
 'aspect ratio',
 'form factor',
 'rectangularity',
 'narrow factor',
 'perimeter ratio of diameter',
 'perimeter ratio of length and width',
 'contrast',
 'correlation',
 'entropy',
 'inverse difference moments']

In [90]:
df = pd.DataFrame(columns=columns)

In [91]:
df

,common name,red_mean,red_var,blue_mean,blue_var,green_mean,green_var,area,perimeter,length,...,aspect ratio,form factor,rectangularity,narrow factor,perimeter ratio of diameter,perimeter ratio of length and width,contrast,correlation,entropy,inverse difference moments


## Create Label Function

In [85]:
def label(img_name):
    num = int(img_name[-8:-4])
    if num >= 1001 and num <= 1059:
        return 'pubescent bamboo'
    elif num >= 1060 and num <= 1122:
        return 'chinese horse chestnut'
    elif num >= 1552 and num <= 1616:
        return 'anhui barberry'
    elif num >= 1123 and num <= 1194:
        return 'chinese redbud'
    elif num >= 1195 and num <= 1267:
        return 'true indigo'
    elif num >= 1268 and num <= 1323:
        return 'japanese maple'
    elif num >= 1324 and num <= 1385:
        return 'nanmu'
    elif num >= 1386 and num <= 1437:
        return 'castor aralia'
    elif num >= 1497 and num <= 1551:
        return 'chinese cinnamon'
    elif num >= 1438 and num <= 1496:
        return 'goldenrain tree'
    elif num >= 2001 and num <= 2050:
        return 'big-fruited holly'
    elif num >= 2051 and num <= 2113:
        return 'japanese cheesewood'
    elif num >= 2114 and num <= 2165:
        return 'wintersweet'
    elif num >= 2166 and num <= 2230:
        return 'camphor tree'
    elif num >= 2231 and num <= 2290:
        return 'japan arrowwood'
    elif num >= 2291 and num <= 2346:
        return 'sweet osmanthus'
    elif num >= 2347 and num <= 2423:
        return 'deodar'
    elif num >= 2424 and num <= 2485:
        return 'gingko'
    elif num >= 2486 and num <= 2546:
        return 'crepe myrtle'
    elif num >= 2547 and num <= 2612:
        return 'oleander'
    elif num >= 2616 and num <= 2675:
        return 'yew plum pine'
    elif num >= 3001 and num <= 3055:
        return 'japanese flowering cherry'
    elif num >= 3056 and num <= 3110:
        return 'glossy privet'
    elif num >= 3111 and num <= 3175:
        return 'chinese toon'
    elif num >= 3176 and num <= 3229:
        return 'peach'
    elif num >= 3230 and num <= 3281:
        return 'ford woodlotus'
    elif num >= 3282 and num <= 3334:
        return 'trident maple'
    elif num >= 3335 and num <= 3389:
        return 'beales barberry'
    elif num >= 3390 and num <= 3446:
        return 'southern magnolia'
    elif num >= 3447 and num <= 3510:
        return 'canadian poplar'
    elif num >= 3511 and num <= 3563:
        return 'chinese tulip tree'
    elif num >= 3566 and num <= 3621:
        return 'tangerine'

## Iterating Over Images and extracting features

In [93]:
index = 0
for i in path:
    # Getting Name of Plant
    name = label(i)
    
    # Getting img
    img = cv2.imread(i)
    
    # Getting Colour Based Features
    # Red
    red = img.copy()
    red[:, :, 0] = 0
    red[:, :, 1] = 0
    red_mean = np.mean(red[:, :, 2])
    red_var = np.std(red[:, :, 2])
    
    # Blue
    blue = img.copy()
    blue[:, :, 1] = 0
    blue[:, :, 2] = 0
    blue_mean = np.mean(blue[:, :, 0])
    blue_var = np.std(blue[:, :, 0])
    
    # Green
    green = img.copy()
    green[:, :, 0] = 0
    green[:, :, 2] = 0
    green_mean = np.mean(green[:, :, 1])
    green_var = np.std(green[:, :, 1])
    
    # Pre-processing for remaining features
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (3, 3), 0)
    thresh, imgBW = cv2.threshold(imgBlur, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    imgInv = cv2.bitwise_not(imgBW)
    kernel = np.ones((50, 50))
    imgClosed = cv2.morphologyEx(imgInv, cv2.MORPH_CLOSE, kernel)
    _, contours, _ = cv2.findContours(imgClosed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    l = 0
    for i in contours:
        if len(i) >l:
            l = len(i)
            cnt = i
    
    # Shape Based Features
    area = cv2.contourArea(cnt)
    per = cv2.arcLength(cnt, True)
    rect = cv2.minAreaRect(cnt)
    (x, y), (w, h), angle = rect
    
    # Morphological Features
    ar = float(h) / w
    ff = (4 * m.pi * area) / (m.pow(per, 2))
    rectangularity = (h * w) / area
    nf = dia / h
    pr = per / dia
    prlw = per / (w + h)
    
    # Texture Based Features
    textures = mt.features.haralick(imgGray)
    textures  = textures.mean(axis=0)
    contrast = textures[1]
    correlation = textures[2]
    entropy = textures[8]
    idf = textures[4]
    
    df = df.append({'common name': name, 'red_mean': red_mean, 'red_var': red_var, 'blue_mean': blue_mean, 'blue_var': blue_var, 'green_mean': green_mean, 'green_var': green_var, 'area': area, 'perimeter': per, 'length': h, 'width': w, 'aspect ratio': ar, 'form factor': ff, 'rectangularity':rectangularity, 'narrow factor': nf, 'perimeter ratio of diameter': pr, 'perimeter ratio of length and width': prlw, 'contrast': contrast, 'correlation': correlation, 'entropy': entropy, 'inverse difference moments': idf}, ignore_index = True)

In [94]:
df

,common name,red_mean,red_var,blue_mean,blue_var,green_mean,green_var,area,perimeter,length,...,aspect ratio,form factor,rectangularity,narrow factor,perimeter ratio of diameter,perimeter ratio of length and width,contrast,correlation,entropy,inverse difference moments
0,pubescent bamboo,233.042456,64.064546,230.722686,70.439153,240.572296,42.356737,198395.0,3549.119297,1635.446899,...,10.006771,0.197924,1.347250,1.071749,2.024842,1.972960,12.639656,0.997666,1.688689,0.911738
1,pubescent bamboo,244.073543,45.414889,242.724323,49.847930,247.329901,32.024069,100889.5,2575.796025,1232.581543,...,10.205549,0.191088,1.475533,1.422046,1.469542,1.903264,8.137424,0.997191,1.193795,0.944818
2,pubescent bamboo,245.420139,43.430021,244.258054,48.248481,248.636254,29.024985,86552.0,2347.269114,1124.194702,...,10.068904,0.197406,1.450186,1.559150,1.339163,1.899323,8.553729,0.996610,0.848758,0.959023
3,pubescent bamboo,234.287985,61.532224,232.272938,67.075823,240.416826,43.685299,191065.5,2973.893566,1384.881470,...,6.922833,0.271482,1.449970,1.265659,1.696665,1.876360,8.440041,0.998419,1.673915,0.914331
4,pubescent bamboo,230.049934,66.982203,226.924000,74.989247,238.373617,44.929827,229856.0,3166.503738,1443.692261,...,5.463408,0.288075,1.659701,1.214101,1.806553,1.853990,8.641438,0.998568,1.968082,0.898644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1902,tangerine,184.260525,93.180342,168.806493,112.210285,215.297724,52.827481,705823.0,4336.717276,1818.690063,...,2.979133,0.471610,1.573011,0.963764,2.474182,1.785270,71.235251,0.993018,5.632665,0.670356
1903,tangerine,185.346329,90.126888,175.594804,102.609698,213.140058,54.705009,727647.0,4438.599526,1874.441284,...,2.943961,0.464129,1.640177,0.935099,2.532308,1.767558,44.850095,0.995506,5.516079,0.670101
1904,tangerine,195.946655,76.084667,178.265622,98.048714,221.792244,44.196328,739384.0,4241.036036,1767.615234,...,2.639542,0.516578,1.600946,0.991612,2.419594,1.740067,70.911466,0.990033,5.773773,0.652729
1905,tangerine,181.371278,95.293325,171.446218,107.993363,208.932923,60.204933,726846.0,4480.918286,1880.785767,...,2.898120,0.454902,1.679268,0.931945,2.556452,1.771287,65.832090,0.994308,5.591044,0.663269


In [96]:
df.to_csv('features.csv')